In [ ]:
from dotenv import load_dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

import overview.config as cfg
import overview.tools.memory as memory
from overview.agent import main_agent, search_subagent
from overview.Analysis.PaperMeta import PaperMetaAnalyzer
from overview.tools.FS import read_from_local
from overview.tools.VectorStore import VectorStore

load_dotenv()


True

In [2]:
vec_store = VectorStore(
    md_dir="./examples/Example/md",
    store_dir="./examples/Example/vectStore",
)

# paper_info = PaperMetaAnalyzer(
#     file_paths=["examples/Example/pdf/1706.03762v7.pdf"]
# ).metadata


sync_store completed


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

sub_model = ChatTongyi(model="qwen-max")
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
agent = main_agent(
    sub_model,
    subagents=[search_subagent(sub_model)],
    tools=[vec_store.retriever_tool, read_from_local],
)


In [4]:
from IPython.display import display, Markdown, HTML
import time


def print_header(role, content, color):
    """打印带颜色的标题（适配黑色背景）"""
    # User 用亮绿色 (#00ff00), AI 用亮青色 (#00ffff)
    html_code = f"""
    <div style="border-left: 5px solid {color}; padding-left: 10px; margin-top: 10px;">
        <span style="color: {color}; font-weight: bold; font-size: 1.1em;">{role}</span>
        <br>
        <span style="color: #cccccc;">{content}</span>
    </div>
    """
    display(HTML(html_code))


def print_ai_markdown(content):
    """使用 Markdown 渲染 AI 的回复（支持公式和代码高亮）"""
    display(Markdown(content))
    display(HTML("<hr style='border-color: #333;'>"))  # 灰色分割线


def parse_ai_response(raw_content):
    if isinstance(raw_content, str):
        return raw_content
    elif isinstance(raw_content, list):
        # 拼接列表中所有的 'text' 字段
        text_parts = []
        for item in raw_content:
            if isinstance(item, dict) and "text" in item:
                text_parts.append(item["text"])
            elif isinstance(item, str):
                text_parts.append(item)
        return "".join(text_parts)
    else:
        return str(raw_content)


# --- 主逻辑 ---

print("🟢 交互开始 (输入 'exit' 结束)")

# 首次提问
question = input("请输入你的问题: ")

if question.strip().lower() not in ["exit", "quit", ""]:
    # 打印用户问题
    print_header("User", question, "#00ff00")  # 亮绿色
    history = cfg.BaseState(messages=[HumanMessage(content=question)])
    # history = memory.resume("history.json")
    while True:
        try:
            # 显示 "思考中..." 提示
            loading_html = HTML(
                '<span style="color: #e6db74; font-style: italic;">🤖 AI 正在思考中...</span>'
            )
            display(loading_html)

            # 1. 调用 AI
            res = agent.invoke(history)
            raw_content = res["messages"][-1].content
            history["messages"] = res["messages"]
            # 2. 【修复点】解析内容，确保它是字符串
            ai_content = parse_ai_response(raw_content)

            # 清除 "思考中" 的提示（这里我们简单地忽略，因为直接打印结果会把它顶上去）

            # 3. 打印 AI 回复
            print_header("AI Assistant", "", "#00ffff")  # 亮青色头
            print_ai_markdown(ai_content)  # Markdown 正文

            # 4. 获取下一次输入
            question = input("请输入你的问题 (exit 退出): ")

            if question.strip().lower() in ["exit", "quit"]:
                print("👋 已退出交互")
                break

            # 打印新一轮的用户问题
            print_header("User", question, "#00ff00")

            history["messages"] += [HumanMessage(content=question)]

        except KeyboardInterrupt:
            print("\n🛑 用户强制中断")
            break
        except Exception as e:
            print(f"❌ 发生错误: {e}")
            if "raw_content" in locals():
                print(f"调试信息: 返回的数据类型是 {type(raw_content)}")
            break
else:
    print("👋 已退出")


🟢 交互开始 (输入 'exit' 结束)


## 问题分析
您询问了论文《Attention Is All You Need》的主要内容。

## 关键发现
- **提出了一种新的网络架构**：Transformer，完全基于注意力机制，摒弃了循环和卷积。
- **实验结果表明**：在两个机器翻译任务上，该模型的质量更优，同时具有更好的并行性，训练时间显著减少。
- **具体成就**：
  - 在WMT 2014英语到德语的翻译任务中，BLEU分数达到28.4，比现有最佳结果高出2分以上。
  - 在WMT 2014英语到法语的翻译任务中，单个模型实现了41.8的BLEU分数，仅用3.5天在八个GPU上完成训练。

## 支持证据
- **摘要**：“我们提出了一个新的简单网络架构，即Transformer，它完全基于注意力机制，彻底摒弃了循环和卷积。实验表明，这些模型在质量上更优，同时具有更好的并行性，并且训练所需的时间显著减少。”
- **第6节 结果**：“在WMT 2014英语到德语的翻译任务中，我们的大Transformer模型（表2中的Transformer (big)）超过了之前报道的所有模型（包括集成模型），BLEU分数提高了2.0以上，达到了28.4的新记录。”

## 额外背景
- Transformer不仅在机器翻译任务上表现出色，还在其他任务上进行了成功应用，如英语成分句法分析，这表明该模型具有很好的泛化能力。

👋 已退出交互


In [ ]:
memory.save_chat("history.json", history)
history

{'messages': [HumanMessage(content='@/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md 这篇论文的主要内容是什么', additional_kwargs={}, response_metadata={}, id='093f9636-458e-4e10-b793-9b2af8711b69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"path": "/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md"}', 'name': 'read_file'}, 'id': 'call_b647f3f7f98f4646bf6117', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '45c97679-d32a-45db-97c4-d780ae85ffb0', 'token_usage': {'input_tokens': 5140, 'output_tokens': 42, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 5182}}, id='lc_run--019b0304-3074-7360-b1a6-de661d6b46d1-0', tool_calls=[{'name': 'read_file', 'args': {'path': '/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md'}, 'id': 'call_b647f3f7f98f4646bf6117', 'type': 'tool_call'}]),
  ToolMessage(c

In [ ]:
state = memory.resume_chat("history.json")

state["messages"] += [HumanMessage(content="我们刚刚聊了什么？")]

agent.invoke(state)

{'messages': [HumanMessage(content='@/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md 这篇论文的主要内容是什么', additional_kwargs={}, response_metadata={}, id='093f9636-458e-4e10-b793-9b2af8711b69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"path": "/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md"}', 'name': 'read_file'}, 'id': 'call_b647f3f7f98f4646bf6117', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '45c97679-d32a-45db-97c4-d780ae85ffb0', 'token_usage': {'input_tokens': 5140, 'output_tokens': 42, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 5182}}, id='lc_run--019b0304-3074-7360-b1a6-de661d6b46d1-0', tool_calls=[{'name': 'read_file', 'args': {'path': '/home/dministrator/git_clone/overview/examples/Example/md/1706.03762v7.md'}, 'id': 'call_b647f3f7f98f4646bf6117', 'type': 'tool_call'}]),
  ToolMessage(c